Here, we test out the partial and full Paper NIFs

In [83]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
# Import all the models, training functions, manipulators here

# Define the relative paths, append it to the system path
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[3]
sys.path.append(str(project_root))

# Import shenanigans
from defs.helper_functions.training_functions import *
from defs.helper_functions.data_manipulation_functions import *
from defs.helper_functions.create_model_and_optimizer_functions import *

# Time, to precisely: time
import time

In [85]:
# Set device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")  # Force CPU for testing purposes
print(f"Using device: {device}")

Using device: cuda


In [86]:
# Import and manipulate all the dataset required

df = pd.read_csv(rf"{project_root}\datasets\airfoil_9k_subset2k.csv") # read csv
df = datareform_tuple_coordinates(df=df, which_coord="y") # take in the y vals
shuffled_df = df.sample(frac=1, random_state=69).reset_index(drop=True) # shuffle the dataframe
input_tensor, target_tensor = seperate_transform(df=df, target_coord=18) # get the target and input tensors
input_tensor = input_tensor.to(device)
target_tensor = target_tensor.to(device)
del df # Clear memory

In [87]:
"""
Trial 1, define configs for Trial 1 here, set device
"""

import torch.nn as nn

cfg_shape_net = {
    'input_dim': 15,
    'output_dim': 3,
    'hidden_units': [8,16,32,16,8],
    'shape_activation': nn.Tanh
}

cfg_param_net = {
    'input_dim': 3,
    'hidden_units': [8,16,16,8],
    'generated_unit': 16,
    'param_activation': nn.Tanh
}

NIF_model, NIF_optimizer = create_NIF_PartialPaper_and_optimizer(cfg_shape_net=cfg_shape_net, cfg_param_net=cfg_param_net, lr=0.00005)
NIF_model = NIF_model.to(device)

In [88]:
# Show the model

from torchviz import make_dot

# Create and move inputs to device
show_tensor = torch.randn(5, cfg_shape_net['input_dim'] + cfg_param_net['input_dim']).to(device)

# Forward pass
output = NIF_model(show_tensor)

make_dot(output, params=dict(NIF_model.named_parameters())).render("model_graph", format="png")

'model_graph.png'

In [89]:
# Now that we have imported data, defined model, and seen what its like, we'll train here


def mse_loss(predictions, targets): return torch.mean((predictions - targets) ** 2)
torch.cuda.synchronize()
start = time.perf_counter()
epoch_train = train_one_epoch(model= NIF_model, optimizer= NIF_optimizer, criterion=mse_loss, inputs= input_tensor, targets= target_tensor, batch_size= 2, device= device, lambda_reg= 0)
torch.cuda.synchronize()
end = time.perf_counter()
print(f"Training took {end - start} seconds")

0.2628285586833954
Rows: 0 to 1 calculating
0.14533725380897522
Rows: 2 to 3 calculating
0.172086700797081
Rows: 4 to 5 calculating
0.3350245952606201
Rows: 6 to 7 calculating
0.47880277037620544
Rows: 8 to 9 calculating
0.5328002572059631
Rows: 10 to 11 calculating
0.3037939667701721
Rows: 12 to 13 calculating
0.2275780290365219
Rows: 14 to 15 calculating
0.21124768257141113
Rows: 16 to 17 calculating
0.48084330558776855
Rows: 18 to 19 calculating
0.46771010756492615
Rows: 20 to 21 calculating
0.32391008734703064
Rows: 22 to 23 calculating
0.45859894156455994
Rows: 24 to 25 calculating
0.1508857011795044
Rows: 26 to 27 calculating
0.31099337339401245
Rows: 28 to 29 calculating
0.6760711669921875
Rows: 30 to 31 calculating
0.39260947704315186
Rows: 32 to 33 calculating
0.47310876846313477
Rows: 34 to 35 calculating
0.23296162486076355
Rows: 36 to 37 calculating
0.38963091373443604
Rows: 38 to 39 calculating
0.6057820320129395
Rows: 40 to 41 calculating
0.5124446749687195
Rows: 42 to 43

In [90]:
# Brief trial to ensure the model updates, we will calculate the MSE for the first row of data

start_idx = 2

trial_input = input_tensor[start_idx:start_idx+1].to(device)

result = NIF_model(trial_input)

trial_target = target_tensor[start_idx:start_idx+1].to(device)
trial_mse = mse_loss(result, trial_target).item()
print("Trial MSE for the first input-target pair:")

print(trial_mse)

Trial MSE for the first input-target pair:
0.037107087671756744
